# Автоматическая суммаризация текста

## Libraries

In [ ]:
!pip install --upgrade pymorphy2[fast] summa sumy lexrank datasets

In [ ]:
!git clone https://github.com/IlyaGusev/summarus
!cd summarus && git checkout f730c89
!cd summarus && pip install -r requirements.txt
%matplotlib inline

In [4]:
!pip install razdel
!pip install navec
!wget https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar

--2022-03-28 12:37:54--  https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53012480 (51M) [application/x-tar]
Saving to: ‘navec_hudlit_v1_12B_500K_300d_100q.tar’

navec_hudlit_v1_12B 100%[===================>]  50.56M  9.95MB/s    in 7.3s    

2022-03-28 12:38:03 (6.96 MB/s) - ‘navec_hudlit_v1_12B_500K_300d_100q.tar’ saved [53012480/53012480]



In [5]:
%cd summarus

/content/summarus


In [6]:
import nltk
from nltk.corpus import stopwords
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


True

## Data

In [11]:
from datasets import load_dataset

gazeta_v1_dataset = load_dataset('IlyaGusev/gazeta')

No config specified, defaulting to: gazeta_dataset/default


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/60964 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6793 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6369 [00:00<?, ? examples/s]

Dataset gazeta_dataset downloaded and prepared to /root/.cache/huggingface/datasets/IlyaGusev___gazeta_dataset/default/2.0.0/e2d171980aa248bc22e0af4f8485ad69071fc8e5f3d54a253c71eb434f6694bd. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
!mkdir -p gazeta_v1
!cd gazeta_v1 && wget https://github.com/IlyaGusev/gazeta/releases/download/1.0/gazeta_jsonl.tar.gz
!cd gazeta_v1 && tar -xzvf gazeta_jsonl.tar.gz

In [ ]:
!rm -f meteor-1.5.tar.gz
!wget -nc https://www.cs.cmu.edu/~alavie/METEOR/download/meteor-1.5.tar.gz
!tar -xzvf meteor-1.5.tar.gz

## Support functions

evaluate and print metrics

In [36]:
# Python wrapper for METEOR implementation, by Xinlei Chen
# Acknowledge Michael Denkowski for the generous discussion and help
# Based on https://github.com/tylin/coco-caption/blob/master/pycocoevalcap/meteor/meteor.py

_BART = "IlyaGusev/mbart_ru_sum_gazeta"
_BERT = "IlyaGusev/rubert_ext_sum_gazeta"
import subprocess
import threading


class Meteor:
    def __init__(self, meteor_jar, language):
        # Used to guarantee thread safety
        self.lock = threading.Lock()

        self.meteor_cmd = ['java', '-jar', '-Xmx2G', meteor_jar, '-', '-', '-stdio', '-l', language, '-norm']
        self.meteor_p = subprocess.Popen(self.meteor_cmd,
                                         stdin=subprocess.PIPE,
                                         stdout=subprocess.PIPE,
                                         stderr=subprocess.STDOUT,
                                         encoding='utf-8',
                                         bufsize=0)

    def compute_score(self, hyps, refs):
        scores = []
        self.lock.acquire()
        for hyp, ref in zip(hyps, refs):
            stat = self._stat(hyp, ref)
            # EVAL ||| stats
            eval_line = 'EVAL ||| {}'.format(" ".join(map(str, map(int, map(float, stat.split())))))
            self.meteor_p.stdin.write('{}\n'.format(eval_line))
            scores.append(float(self.meteor_p.stdout.readline().strip()))
        self.lock.release()

        return sum(scores) / len(scores)

    def _stat(self, hypothesis_str, reference_list):
        # SCORE ||| reference 1 words ||| reference n words ||| hypothesis words
        hypothesis_str = hypothesis_str.replace('|||', '').replace('  ', ' ')
        score_line = ' ||| '.join(('SCORE', ' ||| '.join(reference_list), hypothesis_str))
        self.meteor_p.stdin.write('{}\n'.format(score_line))
        return self.meteor_p.stdout.readline().strip()

    def __del__(self):
        self.lock.acquire()
        self.meteor_p.stdin.close()
        self.meteor_p.kill()
        self.meteor_p.wait()
        self.lock.release()

In [38]:
import os
from collections import Counter
from statistics import mean

from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.chrf_score import corpus_chrf
import torch

def calc_duplicate_n_grams_rate(documents):
    all_ngrams_count = Counter()
    duplicate_ngrams_count = Counter()
    for doc in documents:
        words = doc.split(" ")
        for n in range(1, 5):
            ngrams = [tuple(words[i:i+n]) for i in range(len(words)-n+1)]
            unique_ngrams = set(ngrams)
            all_ngrams_count[n] += len(ngrams)
            duplicate_ngrams_count[n] += len(ngrams) - len(unique_ngrams)
    return {n: duplicate_ngrams_count[n]/all_ngrams_count[n] if all_ngrams_count[n] else 0.0
            for n in range(1, 5)}


def calc_bert_score(
    hyps,
    refs,
    lang="ru",
    bert_score_model=None,
    num_layers=None,
    idf=False,
    batch_size=32
):
    import bert_score
    all_preds, hash_code = bert_score.score(
        hyps,
        refs,
        lang=lang,
        model_type=bert_score_model,
        num_layers=num_layers,
        verbose=False,
        idf=idf,
        batch_size=batch_size,
        return_hash=True
    )
    avg_scores = [s.mean(dim=0) for s in all_preds]
    return {
        "p": avg_scores[0].cpu().item(),
        "r": avg_scores[1].cpu().item(),
        "f": avg_scores[2].cpu().item()
    }, hash_code


def calc_metrics(
    refs, hyps,
    language,
    metric="all",
    meteor_jar=None
):
    metrics = dict()
    metrics["count"] = len(hyps)
    metrics["ref_example"] = refs[-1]
    metrics["hyp_example"] = hyps[-1]
    many_refs = [[r] if r is not list else r for r in refs]
    if metric in ("bleu", "all"):
        t_hyps = [hyp.split(" ") for hyp in hyps]
        t_refs = [[r.split(" ") for r in rs] for rs in many_refs]
        metrics["bleu"] = corpus_bleu(t_refs, t_hyps)
    if metric in ("rouge", "all"):
        rouge = Rouge()
        scores = rouge.get_scores(hyps, refs, avg=True)
        metrics.update(scores)
    if metric in ("meteor", "all") and meteor_jar is not None and os.path.exists(meteor_jar):
        meteor = Meteor(meteor_jar, language=language)
        metrics["meteor"] = meteor.compute_score(hyps, many_refs)
    if metric in ("duplicate_ngrams", "all"):
        metrics["duplicate_ngrams"] = dict()
        metrics["duplicate_ngrams"].update(calc_duplicate_n_grams_rate(hyps))
    if metric in ("bert_score",) and torch.cuda.is_available():
        bert_scores, hash_code = calc_bert_score(hyps, refs)
        metrics["bert_score_{}".format(hash_code)] = bert_scores
    if metric in ("chrf", "all"):
        metrics["chrf"] = corpus_chrf(refs, hyps, beta=1.0)
    if metric in ("length", "all"):
        metrics["length"] = mean([len(h) for h in hyps])
    return metrics


def print_metrics(refs, hyps, language, metric="all", meteor_jar=None):
    metrics = calc_metrics(refs, hyps, language=language, metric=metric, meteor_jar=meteor_jar)

    print("-------------METRICS-------------")
    print("Count:\t", metrics["count"])
    print("Ref:\t", metrics["ref_example"])
    print("Hyp:\t", metrics["hyp_example"])

    if "bleu" in metrics:
        print("BLEU:     \t{:3.1f}".format(metrics["bleu"] * 100.0))
    if "chrf" in metrics:
        print("chrF:     \t{:3.1f}".format(metrics["chrf"] * 100.0))
    if "rouge-1" in metrics:
        print("ROUGE-1-F:\t{:3.1f}".format(metrics["rouge-1"]['f'] * 100.0))
        print("ROUGE-2-F:\t{:3.1f}".format(metrics["rouge-2"]['f'] * 100.0))
        print("ROUGE-L-F:\t{:3.1f}".format(metrics["rouge-l"]['f'] * 100.0))
    if "meteor" in metrics:
        print("METEOR:   \t{:3.1f}".format(metrics["meteor"] * 100.0))
    if "duplicate_ngrams" in metrics:
        print("Dup 1-grams:\t{:3.1f}".format(metrics["duplicate_ngrams"][1] * 100.0))
        print("Dup 2-grams:\t{:3.1f}".format(metrics["duplicate_ngrams"][2] * 100.0))
        print("Dup 3-grams:\t{:3.1f}".format(metrics["duplicate_ngrams"][3] * 100.0))
    if "length" in metrics:
        print("Avg length:\t{:3.1f}".format(metrics["length"]))
    for key, value in metrics.items():
        if "bert_score" not in key:
            continue
        print("{}:\t{:3.1f}".format(key, value["f"] * 100.0))

ModuleNotFoundError: ignored

In [ ]:
def postprocess(ref, hyp, language, is_multiple_ref=False, detokenize_after=False, tokenize_after=False, lower=False):
    if is_multiple_ref:
        reference_sents = ref.split(" s_s ")
        decoded_sents = hyp.split("s_s")
        hyp = [w.replace("<", "&lt;").replace(">", "&gt;").strip() for w in decoded_sents]
        ref = [w.replace("<", "&lt;").replace(">", "&gt;").strip() for w in reference_sents]
        hyp = " ".join(hyp)
        ref = " ".join(ref)
    ref = ref.strip()
    hyp = hyp.strip()
    if detokenize_after:
        hyp = punct_detokenize(hyp)
        ref = punct_detokenize(ref)
    if tokenize_after:
        hyp = hyp.replace("@@UNKNOWN@@", "<unk>")
        if language == "ru":
            hyp = " ".join([token.text for token in razdel.tokenize(hyp)])
            ref = " ".join([token.text for token in razdel.tokenize(ref)])
        else:
            hyp = " ".join([token for token in nltk.word_tokenize(hyp)])
            ref = " ".join([token for token in nltk.word_tokenize(ref)])
    if lower:
        hyp = hyp.lower()
        ref = ref.lower()
    return ref, hyp

In [ ]:
from tqdm.notebook import tqdm

def calc_method_score(records, predict_func, nrows=None, meteor_jar="meteor-1.5/meteor-1.5.jar"):
    references = []
    predictions = []
    for i, record in enumerate(tqdm(records)):
        if nrows is not None and i >= nrows:
            break
        references.append(record["summary"])
        predictions.append(predict_func(record["text"], record["summary"]))

    for i, (ref, hyp) in enumerate(tqdm(zip(references, predictions))):
        references[i], predictions[i] = postprocess(ref, hyp, language="ru", tokenize_after=True, lower=True)
    print_metrics(references, predictions, language="ru", meteor_jar=meteor_jar)


def calc_bert_score(records, predict_func, nrows=None):
    references = []
    predictions = []
    for i, record in enumerate(tqdm(records)):
        if nrows is not None and i >= nrows:
            break
        references.append(record["summary"])
        predictions.append(predict_func(record["text"], record["summary"]))

    for i, (ref, hyp) in enumerate(tqdm(zip(references, predictions))):
        references[i], predictions[i] = postprocess(ref, hyp, language="ru", tokenize_after=False, lower=False)
    print_metrics(references, predictions, language="ru", meteor_jar=None, metric="bert_score")

## Baseline - Lead

First sentences of a text as a baseline.

In [ ]:
import razdel

def predict_lead(text, summary, n):
    sentences = [sentence.text for sentence in razdel.sentenize(text)]
    prediction = " ".join(sentences[:n])
    return prediction

In [ ]:
# Gazeta v1, Lead-1
calc_method_score(gazeta_v1_dataset["test"], lambda x, y: predict_lead(x, y, 1))

  0%|          | 0/5770 [00:00<?, ?it/s]

0it [00:00, ?it/s]

-------------METRICS-------------
Count:	 5770
Ref:	 россия пригласила китай принять участие в тендере на строительство нового класса атомного ледокола , что обязательно потребует разработки реакторов — и пекин воспользуется этим для создания собственного атомного авианосца . по данным аналитиков , китайцы хотят спустить на воду авианосец , сопоставимый по размерам и характеристикам с американскими кораблями типа gerald r . ford .
Hyp:	 китай создает свой собственный атомный авианосец с использованием российских технических новшеств и ноу-хау в сфере ядерного оружия , сообщает американское аналитическое издание the national interest .
BLEU:     	5.3
chrF:     	32.2
ROUGE-1-F:	27.6
ROUGE-2-F:	12.9
ROUGE-L-F:	20.2
METEOR:   	18.6
Dup 1-grams:	6.0
Dup 2-grams:	0.1
Dup 3-grams:	0.0
Avg length:	158.2


In [ ]:
# Gazeta v1, Lead-1
calc_method_score(gazeta_v1_dataset["test"], lambda x, y: predict_lead(x, y, 1))

-------------METRICS-------------
Count:	 5770
Ref:	 россия пригласила китай принять участие в тендере на строительство нового класса атомного ледокола , что обязательно потребует разработки реакторов — и пекин воспользуется этим для создания собственного атомного авианосца . по данным аналитиков , китайцы хотят спустить на воду авианосец , сопоставимый по размерам и характеристикам с американскими кораблями типа gerald r . ford .
Hyp:	 китай создает свой собственный атомный авианосец с использованием российских технических новшеств и ноу-хау в сфере ядерного оружия , сообщает американское аналитическое издание the national interest .
BLEU:     	5.3
chrF:     	32.2
ROUGE-1-F:	27.6
ROUGE-2-F:	12.9
ROUGE-L-F:	20.2
METEOR:   	18.6
Dup 1-grams:	6.0
Dup 2-grams:	0.1
Dup 3-grams:	0.0
Avg length:	158.2


In [ ]:
# Gazeta v2, Lead-1
calc_method_score(gazeta_v2_dataset["test"], lambda x, y: predict_lead(x, y, 1))

-------------METRICS-------------
Count:	 6793
Ref:	 токио пока не дал гарантий москве , что не станет размещать на своей территории американские ракеты , заявил владимир путин . это , по его словам , препятствует заключению мирного соглашения с японией . его отсутствие президент назвал нонсенсом .
Hyp:	 москва никогда не отказывалась обсуждать с токио мирный договор , который так и не был заключен между двумя странами по итогам второй мировой войны .
BLEU:     	3.7
chrF:     	29.1
ROUGE-1-F:	23.3
ROUGE-2-F:	9.1
ROUGE-L-F:	16.7
METEOR:   	15.0
Dup 1-grams:	6.1
Dup 2-grams:	0.2
Dup 3-grams:	0.0
Avg length:	162.2


In [ ]:
# Gazeta v1, Lead-3
calc_method_score(gazeta_v1_dataset["test"], lambda x, y: predict_lead(x, y, 3))

-------------METRICS-------------
Count:	 5770
Ref:	 россия пригласила китай принять участие в тендере на строительство нового класса атомного ледокола , что обязательно потребует разработки реакторов — и пекин воспользуется этим для создания собственного атомного авианосца . по данным аналитиков , китайцы хотят спустить на воду авианосец , сопоставимый по размерам и характеристикам с американскими кораблями типа gerald r . ford .
Hyp:	 китай создает свой собственный атомный авианосец с использованием российских технических новшеств и ноу-хау в сфере ядерного оружия , сообщает американское аналитическое издание the national interest . китай уже изучает ядерные реакторы на крупнейших ледоколах россии . в частности , москва пригласила пекин принять участие в тендере на строительство нового класса атомного ледокола , что обязательно потребует разработки реакторов .
BLEU:     	10.8
chrF:     	38.5
ROUGE-1-F:	31.0
ROUGE-2-F:	13.4
ROUGE-L-F:	26.3
METEOR:   	26.0
Dup 1-grams:	18.4
Dup 2-grams

In [ ]:
# Gazeta v2, Lead-3
calc_method_score(gazeta_v2_dataset["test"], lambda x, y: predict_lead(x, y, 3))

-------------METRICS-------------
Count:	 6793
Ref:	 токио пока не дал гарантий москве , что не станет размещать на своей территории американские ракеты , заявил владимир путин . это , по его словам , препятствует заключению мирного соглашения с японией . его отсутствие президент назвал нонсенсом .
Hyp:	 москва никогда не отказывалась обсуждать с токио мирный договор , который так и не был заключен между двумя странами по итогам второй мировой войны . об этом заявил президент владимир путин на пленарном заседании восточного экономического форума ( вэф ) . трансляцию ведет телеканал « россия 24 » .
BLEU:     	7.9
chrF:     	36.0
ROUGE-1-F:	27.2
ROUGE-2-F:	10.1
ROUGE-L-F:	22.8
METEOR:   	22.0
Dup 1-grams:	18.7
Dup 2-grams:	1.6
Dup 3-grams:	0.4
Avg length:	429.1


## Статистические модели

In [14]:
import pymorphy2
import numpy as np
import math 
import razdel

nltk.download('stopwords')
rus_stopwords = stopwords.words('russian')

morph = pymorphy2.MorphAnalyzer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


### Экстрактивная суммаризация на основе близости предложений (LexRank)

Оцениваем близость предложений как косинусную близость между TF-IDF векторами. 
Предполагаем, что предложение, которое наиболее близко ко всем остальным - больше остальных описывает весь текст в совокупности

https://cyberleninka.ru/article/n/obzor-zadachi-avtomaticheskoy-summarizatsii-teksta/viewer

https://iq.opengenus.org/lexrank-text-summarization/

TF-IDF - https://habr.com/ru/company/Voximplant/blog/446738/

In [18]:
import collections

class LexRankSummarizer():

    def __init__(self, stopwords, summary_part=0.1):
        self._stop_words = frozenset(rus_stopwords)
        self.epsilon = 1e-4
        self.threshold = 0.1
        self.summary_part = summary_part

    def stop_words(self):
        return self._stop_words

    def __call__(self, document):

        sentences_words = [[morph.parse(token.text)[0].normal_form for token in razdel.tokenize(sentence.text) if token.text.lower() not in self._stop_words] for sentence in razdel.sentenize(document)]
        sentences = [sentence.text for sentence in razdel.sentenize(document)]

        tf_metrics = self._compute_tf(sentences_words)
        idf_metrics = self._compute_idf(sentences_words)

        matrix = self._create_matrix(sentences_words, self.threshold, tf_metrics, idf_metrics)
        ranks = self.power_method(matrix, self.epsilon)
        
        return self._get_best_sentences({i: rank for i, rank in enumerate(ranks)}, sentences)

    def _compute_tf(self, sentences):
        tf_metrics = []
        for sentence in sentences:
            metrics = {}
        #На вход берем текст в виде списка (list) слов
        #Считаем частотность всех терминов во входном массиве с помощью 
        #метода Counter библиотеки collections
            tf_text = collections.Counter(sentence)
            for i, term in enumerate(sentence):
                metrics[term] = tf_text[i] / float(len(sentence))
                #для каждого слова в tf_text считаем TF путём деления
                #встречаемости слова на общее количество слов в тексте
            #возвращаем объект типа Counter c TF всех слов текста
            tf_metrics.append(metrics)
        return tf_metrics
    
    def _compute_idf(self, sentences):
      idf_metrics = {}
      for sentence in sentences:
          for i, term in enumerate(sentence):
              idf_metrics[term] = math.log10(len(sentences)/sum([1.0 for _sent in sentences if term in _sent]))
      return idf_metrics

    def _create_matrix(self, sentences, threshold, tf_metrics, idf_metrics):
        """
        Creates matrix of shape |sentences|×|sentences|.
        """
        # create matrix |sentences|×|sentences| filled with zeroes
        sentences_count = len(sentences)
        matrix = np.zeros((sentences_count, sentences_count))
        degrees = np.zeros((sentences_count, ))

        for row, (sentence1, tf1) in enumerate(zip(sentences, tf_metrics)):
            for col, (sentence2, tf2) in enumerate(zip(sentences, tf_metrics)):
                matrix[row, col] = self.cosine_similarity(sentence1, sentence2, tf1, tf2, idf_metrics)

                if matrix[row, col] > threshold:
                    matrix[row, col] = 1.0
                    degrees[row] += 1
                else:
                    matrix[row, col] = 0

        for row in range(sentences_count):
            for col in range(sentences_count):
                if degrees[row] == 0:
                    degrees[row] = 1

                matrix[row][col] = matrix[row][col] / degrees[row]

        return matrix

    @staticmethod
    def cosine_similarity(sentence1, sentence2, tf1, tf2, idf_metrics):
        """
        We compute idf-modified-cosine(sentence1, sentence2) here.
        It's cosine similarity of these two sentences (vectors) A, B computed as cos(x, y) = A . B / (|A| . |B|)
        Sentences are represented as vector TF*IDF metrics.
        :param sentence1:
            Iterable object where every item represents word of 1st sentence.
        :param sentence2:
            Iterable object where every item represents word of 2nd sentence.
        :type tf1: dict
        :param tf1:
            Term frequencies of words from 1st sentence.
        :type tf2: dict
        :param tf2:
            Term frequencies of words from 2nd sentence
        :type idf_metrics: dict
        :param idf_metrics:
            Inverted document metrics of the sentences. Every sentence is treated as document for this algorithm.
        :rtype: float
        :return:
            Returns -1.0 for opposite similarity, 1.0 for the same sentence and zero for no similarity between sentences.
        """
        unique_words1 = frozenset(sentence1)
        unique_words2 = frozenset(sentence2)
        common_words = unique_words1 & unique_words2

        numerator = 0.0
        for term in common_words:
            numerator += tf1[term]*tf2[term] * idf_metrics[term]**2

        denominator1 = sum((tf1[t]*idf_metrics[t])**2 for t in unique_words1)
        denominator2 = sum((tf2[t]*idf_metrics[t])**2 for t in unique_words2)

        if denominator1 > 0 and denominator2 > 0:
            return numerator / (math.sqrt(denominator1) * math.sqrt(denominator2))
        else:
            return 0.0

    @staticmethod
    def power_method(matrix, epsilon):
        transposed_matrix = matrix.T
        sentences_count = len(matrix)
        p_vector = np.array([1.0 / sentences_count] * sentences_count)
        lambda_val = 1.0

        while lambda_val > epsilon:
            next_p = np.dot(transposed_matrix, p_vector)
            lambda_val = np.linalg.norm(np.subtract(next_p, p_vector))
            p_vector = next_p

        return p_vector
    
    def _get_best_sentences(self, ratings, sentences):
        k = round(self.summary_part * len(sentences))
        sorted_ind = sorted([i for i, _ in sorted(ratings.items(), key=lambda item: item[1], reverse=True)][:k])
        return " ".join([sentences[i] for i in sorted_ind])

In [19]:
LexRank = LexRankSummarizer(rus_stopwords)

def predict_lex_rank(text, summary):
    return LexRank(text)

In [ ]:
calc_method_score(gazeta_v1_dataset["test"], predict_lex_rank)

  0%|          | 0/5770 [00:00<?, ?it/s]

0it [00:00, ?it/s]

-------------METRICS-------------
Count:	 5770
Ref:	 россия пригласила китай принять участие в тендере на строительство нового класса атомного ледокола , что обязательно потребует разработки реакторов — и пекин воспользуется этим для создания собственного атомного авианосца . по данным аналитиков , китайцы хотят спустить на воду авианосец , сопоставимый по размерам и характеристикам с американскими кораблями типа gerald r . ford .
Hyp:	 китай создает свой собственный атомный авианосец с использованием российских технических новшеств и ноу-хау в сфере ядерного оружия , сообщает американское аналитическое издание the national interest . китай уже изучает ядерные реакторы на крупнейших ледоколах россии . в частности , москва пригласила пекин принять участие в тендере на строительство нового класса атомного ледокола , что обязательно потребует разработки реакторов .
BLEU:     	9.5
chrF:     	37.8
ROUGE-1-F:	30.9
ROUGE-2-F:	13.3
ROUGE-L-F:	25.8
METEOR:   	26.5
Dup 1-grams:	20.8
Dup 2-grams:

In [ ]:
calc_method_score(gazeta_v2_dataset["test"], predict_lex_rank)

  0%|          | 0/6793 [00:00<?, ?it/s]

0it [00:00, ?it/s]

-------------METRICS-------------
Count:	 6793
Ref:	 токио пока не дал гарантий москве , что не станет размещать на своей территории американские ракеты , заявил владимир путин . это , по его словам , препятствует заключению мирного соглашения с японией . его отсутствие президент назвал нонсенсом .
Hyp:	 москва никогда не отказывалась обсуждать с токио мирный договор , который так и не был заключен между двумя странами по итогам второй мировой войны . об этом заявил президент владимир путин на пленарном заседании восточного экономического форума ( вэф ) . трансляцию ведет телеканал « россия 24 » .
BLEU:     	7.2
chrF:     	35.4
ROUGE-1-F:	27.0
ROUGE-2-F:	9.9
ROUGE-L-F:	22.3
METEOR:   	22.1
Dup 1-grams:	20.5
Dup 2-grams:	1.9
Dup 3-grams:	0.5
Avg length:	484.1


### LSA for text summarization

Данное матричное разложение TF-IDF векторных представлений предложений дает крайнюю правую матрицу, которая показывает какое из предложений лучше других описывает определенную скрытую тему. Берем как сокращение количество предложений равное количеству скрытых тем. Для каждой темы выбираем предложение, которое лучше остальных описывает эту тему.

https://cyberleninka.ru/article/n/obzor-zadachi-avtomaticheskoy-summarizatsii-teksta/viewer

LSA - https://habr.com/ru/post/240209/

TF-IDF - https://habr.com/ru/company/Voximplant/blog/446738/


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

class LSA_Summarizer():
  def __init__(self, stopwords, vectorizer, summary_part=0.1):
    self.vectorizer = vectorizer
    self.summary_part = summary_part
    self._stop_words = frozenset(stopwords)

  def stop_words(self):
        return self._stop_words

  def __call__(self, document):
    sentences = [sentence.text for sentence in razdel.sentenize(document)]
    clear_sentences = [' '.join([morph.parse(token.text)[0].normal_form for token in razdel.tokenize(sentence.text) if token.text.lower() not in rus_stopwords]) for sentence in razdel.sentenize(document)]
    X = vectorizer.fit_transform(clear_sentences)
    svd = TruncatedSVD(n_components=round(self.summary_part*len(clear_sentences) + 0.49), n_iter=7, random_state=42)
    V = svd.fit_transform(X) # n_sentences, n_topics
    sorted_ind = sorted(np.unique(np.argmax(V, axis=0), axis=0))
    return " ".join([sentences[i] for i in sorted_ind])

In [ ]:
vectorizer = TfidfVectorizer()
LSA = LSA_Summarizer(rus_stopwords, vectorizer)

def predict_lsa(text, summary):
    return LSA(text)

In [ ]:
calc_method_score(gazeta_v1_dataset["test"], predict_lsa)

  0%|          | 0/5770 [00:00<?, ?it/s]

0it [00:00, ?it/s]

-------------METRICS-------------
Count:	 5770
Ref:	 россия пригласила китай принять участие в тендере на строительство нового класса атомного ледокола , что обязательно потребует разработки реакторов — и пекин воспользуется этим для создания собственного атомного авианосца . по данным аналитиков , китайцы хотят спустить на воду авианосец , сопоставимый по размерам и характеристикам с американскими кораблями типа gerald r . ford .
Hyp:	 китай уже изучает ядерные реакторы на крупнейших ледоколах россии . корабль впервые был спущен на воду 26 апреля 2017 года . по мнению китайских экспертов , чтобы быть конкурентоспособным , пекину необходим более мощный атомный авианосец . предполагается , что к середине 2030-х в распоряжении пекина могут быть шесть авианосцев .
BLEU:     	4.9
chrF:     	31.4
ROUGE-1-F:	22.5
ROUGE-2-F:	6.8
ROUGE-L-F:	18.5
METEOR:   	17.9
Dup 1-grams:	22.1
Dup 2-grams:	2.6
Dup 3-grams:	0.8
Avg length:	494.8


In [ ]:
calc_method_score(gazeta_v2_dataset["test"], predict_lsa)

  0%|          | 0/6793 [00:00<?, ?it/s]

0it [00:00, ?it/s]

-------------METRICS-------------
Count:	 6793
Ref:	 токио пока не дал гарантий москве , что не станет размещать на своей территории американские ракеты , заявил владимир путин . это , по его словам , препятствует заключению мирного соглашения с японией . его отсутствие президент назвал нонсенсом .
Hyp:	 москва никогда не отказывалась обсуждать с токио мирный договор , который так и не был заключен между двумя странами по итогам второй мировой войны . эти вопросы поставлены перед японской стороной , ответа мы пока не получили . когда именно заработает новый налоговый режим , не уточнялось .
BLEU:     	4.1
chrF:     	31.0
ROUGE-1-F:	21.4
ROUGE-2-F:	5.8
ROUGE-L-F:	17.4
METEOR:   	16.6
Dup 1-grams:	21.9
Dup 2-grams:	2.7
Dup 3-grams:	0.8
Avg length:	492.4


Пример:

In [ ]:
gazeta_v2_dataset["test"]['text'][0]

'На этих выходных в Берлине прошли крупные акции протеста против введенных для борьбы с коронавирусом ограничений. Демонстранты скандировали «Путин!» По словам депутата городской палаты представителей Гуннара Линдеманна («Альтернатива для Германии»), люди выкрикивали фамилию российского президента из уважения к нему. В комментарии РИА «Новости» немецкий политик отметил, что среди населения Германии Владимир Путин имеет хорошую репутацию. По его мнению, протестующие ранее пришли к российскому посольству, чтобы «привлечь внимание к условиям в Германии», надеясь, что Россия сможет оказать влияние на канцлера ФРГ Ангелу Меркель. «На мой взгляд, опасности для посольства России не возникло ни разу», — сказал депутат. Несмотря на то что протест оказался массовым, выступления носили «преимущественно мирный характер», уверен Линдеманн. По его словам, исключением стала только ситуацию у немецкого парламента. Там «несколько странных участников демонстрации попытались штурмовать бундестаг в знак п

In [ ]:
gazeta_v2_dataset["test"]['summary'][10]

'Лишний вес при COVID-19 повышает риск столкнуться с осложнениями и оказаться на ИВЛ, предупреждают французские врачи — ожирение наблюдается почти у всех пациентов с коронавирусом, попавших в отделения интенсивной терапии. И чем выше вес, тем выше и вероятность пострадать от тяжелого течения болезни.'

In [ ]:
LSA(gazeta_v2_dataset["test"]['text'][10])

'Оказалось, что среди пациентов с COVID-19 почти половина страдала от ожирения, у четверти оно было тяжелым (ИМТ>35). У половины наблюдался избыточный вес, у оставшейся четверти масса тела была в норме. Также им почти в два раза чаще требовалась интенсивная терапия. Последний факт снижает эффективность искусственной вентиляции легких, отмечают врачи — жир сдавливает легкие, мешая доступу воздуха.'

### Экстрактивная суммаризация на основе вхождения общих слов (TextRank)

Оцениваем близость предложений как количество общих слов (смотри формулу, там не совсем так просто). 
Предполагаем, что предложение, которое наиболее близко ко всем остальным - больше остальных описывает весь текст в совокупности

TextRank - https://cyberleninka.ru/article/n/obzor-zadachi-avtomaticheskoy-summarizatsii-teksta/viewer

Экстрактивная суммаризация на основе вхождения общих слов
https://habr.com/ru/post/514540/

https://iq.opengenus.org/textrank-for-text-summarization/

In [ ]:
class TextRankSummarizer():

    def __init__(self, stopwords, summary_part=0.1):
        self._stop_words = frozenset(rus_stopwords)
        self._delta = 1e-7 
        self.epsilon = 1e-4
        self.damping = 0.85
        self.summary_part = summary_part

    def stop_words(self):
        return self._stop_words

    def __call__(self, document):
        ratings, sentences = self.rate_sentences(document)
        return self._get_best_sentences(ratings, sentences)

    def _create_matrix(self, document):
        """Create a stochastic matrix for TextRank. 
        Element at row i and column j of the matrix corresponds to the 
        similarity of sentence i and j, where the similarity is computed 
        as the number of common words between them, divided by their sum of 
        logarithm of their lengths. After such matrix is created, it is turned 
        into a stochastic matrix by normalizing over columns i.e. making the 
        columns sum to one. TextRank uses PageRank algorithm with damping, 
        so a damping factor is incorporated as explained in TextRank's paper. 
        The resulting matrix is a stochastic matrix ready for power method."""
        sentences_as_words = [[morph.parse(token.text)[0].normal_form for token in razdel.tokenize(sentence.text) if token.text.lower() not in self._stop_words] for sentence in razdel.sentenize(document)]
        sentences_count = len(sentences_as_words) 
        weights = np.zeros((sentences_count, sentences_count))

        for i, words_i in enumerate(sentences_as_words):
            for j, words_j in enumerate(sentences_as_words):
                weights[i, j] = self._rate_sentences_edge(words_i, words_j)
        weights /= (weights.sum(axis=1)[:, np.newaxis] + self._delta) # delta added to prevent zero-division error 
    
        return np.full((sentences_count, sentences_count), (1. - self.damping) / sentences_count) + self.damping * weights

    def rate_sentences(self, document):
        matrix = self._create_matrix(document)
        ranks = self.power_method(matrix)

        sentences = [sentence.text for sentence in razdel.sentenize(document)]
        return {i: rank for i, rank in enumerate(ranks)}, sentences
    
    @staticmethod
    def _rate_sentences_edge(sentence1, sentence2):
        rank = 0
        for w1 in sentence1:
            for w2 in sentence2:
                rank += int(w1 == w2)

        if rank == 0:
            return 0.0

        assert len(sentence1) > 0 and len(sentence2) > 0

        norm = math.log(len(sentence1)) + math.log(len(sentence2))
        if np.isclose(norm, 0.):
            # This should only happen when sentence1 and sentence2 only have a single sentence. Thus, rank can only be 0 or 1.
            assert rank in (0, 1)
            return rank * 1.0
        else:
            return rank / norm

    def power_method(self, matrix):
        transposed_matrix = matrix.T
        sentences_count = len(matrix)
        p_vector = np.array([1.0 / sentences_count] * sentences_count)
        lambda_val = 1.0

        while lambda_val > self.epsilon:
            next_p = np.dot(transposed_matrix, p_vector)
            lambda_val = np.linalg.norm(np.subtract(next_p, p_vector))
            p_vector = next_p

        return p_vector
    
    def _get_best_sentences(self, ratings, sentences):
        k = round(self.summary_part * len(sentences))
        sorted_ind = sorted([i for i, _ in sorted(ratings.items(), key=lambda item: item[1], reverse=True)][:k])
        return " ".join([sentences[i] for i in sorted_ind])

In [ ]:
TextRank = TextRankSummarizer(rus_stopwords)

def predict_text_rank(text, summary):
    return TextRank(text)

In [ ]:
calc_method_score(gazeta_v1_dataset["test"], predict_text_rank)

  0%|          | 0/5770 [00:00<?, ?it/s]

0it [00:00, ?it/s]

-------------METRICS-------------
Count:	 5770
Ref:	 россия пригласила китай принять участие в тендере на строительство нового класса атомного ледокола , что обязательно потребует разработки реакторов — и пекин воспользуется этим для создания собственного атомного авианосца . по данным аналитиков , китайцы хотят спустить на воду авианосец , сопоставимый по размерам и характеристикам с американскими кораблями типа gerald r . ford .
Hyp:	 « этот подход отличается от того , как соединенные штаты и франция разработали ядерные реакторы для своих крупнейших судов , но , вероятно , представляет собой лучшую тактику для китая на данный момент » , — пишет автор статьи роберт фарли . покрытие , как отмечают местные журналисты , может эффективно снижать импульс садящегося на палубу истребителя на скоростях до 300 км / ч . кроме того , оно обеспечивает качественное противоскольжение при взлете воздушных судов с борта авианосца . судно , названное , предположительно , в честь рака-богомола pipixia 

In [ ]:
calc_method_score(gazeta_v2_dataset["test"], predict_text_rank)

  0%|          | 0/6793 [00:00<?, ?it/s]

0it [00:00, ?it/s]

-------------METRICS-------------
Count:	 6793
Ref:	 токио пока не дал гарантий москве , что не станет размещать на своей территории американские ракеты , заявил владимир путин . это , по его словам , препятствует заключению мирного соглашения с японией . его отсутствие президент назвал нонсенсом .
Hyp:	 путин подчеркнул , что и россия , и япония заинтересованы в « абсолютной » нормализации двусторонних отношений , поскольку у стран есть интерес для дальнейшего развития стратегического сотрудничества . президент выразил надежду , что новый налоговый режим на курилах даст результаты для развития островов — в первую очередь , в отраслях туризма , морского промысла и переработки . премьер-министр михаил мишустин уверял , что курильский офшор не будет аналогом внутренних офшоров в калмыкии , ингушетии , на алтае и других российских регионах , которые создавались в 90-е годы прошлого века .
BLEU:     	2.2
chrF:     	26.2
ROUGE-1-F:	17.3
ROUGE-2-F:	4.0
ROUGE-L-F:	12.7
METEOR:   	13.7
Dup 1-g

### KL Sum algorithm for text summarization

Представляем входной текст, как некоторое распределение слов. Подсчитываем для этой цели все частоты. Затем считаем, что каждое предложение - тоже некоторое распределение слов. Подсчитываем в них частоты.

Затем:
* Выбираем предложение, у которого распределение слов имеет наименьшую KL-дивергенцию с исходным текстом, и добавляем его в сокращение.
* Добираем по такому принципу еще предложения, пока их не останется.
* Выбираем первые k предложений (сокращение не может быть больше 10% исходного текста), что мы добавили. Они и дают большую сходимость распределения сокращения с распределением исходного текста

KL Divergence - https://iq.opengenus.org/kl-divergence/

https://iq.opengenus.org/k-l-sum-algorithm-for-text-summarization/

In [15]:
class KLSummarizer():
    """
    Method that greedily adds sentences to a summary so long as it decreases the
    KL Divergence.
    """
    def __init__(self, stopwords, summary_part=0.1):
        self.stop_words = frozenset(stopwords)
        self.summary_part = summary_part

    def __call__(self, document):
        sentences = [sentence.text for sentence in razdel.sentenize(document)]
        ratings = self._compute_ratings(sentences)

        return self._get_best_sentences(ratings, sentences)

    def _get_all_words_in_doc(self, sentences):
        return [morph.parse(token.text)[0].normal_form for sentence in sentences for token in razdel.tokenize(sentence) if token.text.lower() not in self.stop_words]

    def _get_content_words_in_sentence(self, sentence):
        return [morph.parse(token.text)[0].normal_form for token in razdel.tokenize(sentence) if token.text.lower() not in self.stop_words]

    @staticmethod
    def _compute_word_freq(list_of_words):
        word_freq = {}
        for w in list_of_words:
            word_freq[w] = word_freq.get(w, 0) + 1
        return word_freq

    def compute_tf(self, sentences):
        """
        Computes the normalized term frequency as explained in http://www.tfidf.com/
        """
        content_words = self._get_all_words_in_doc(sentences)
        content_words_count = len(content_words)
        content_words_freq = self._compute_word_freq(content_words)
        content_word_tf = dict((w, f / content_words_count) for w, f in content_words_freq.items())
        return content_word_tf

    def _joint_freq(self, word_list_1, word_list_2):
        # combined length of the word lists
        total_len = len(word_list_1) + len(word_list_2)

        # word frequencies within each list
        wc1 = self._compute_word_freq(word_list_1)
        wc2 = self._compute_word_freq(word_list_2)

        # inputs the counts from the first list
        joint = wc1.copy()

        # adds in the counts of the second list
        for k in wc2:
            if k in joint:
                joint[k] += wc2[k]
            else:
                joint[k] = wc2[k]

        # divides total counts by the combined length
        for k in joint:
            joint[k] /= float(total_len)

        return joint

    @staticmethod
    def _kl_divergence(summary_freq, doc_freq):
        """
        Note: Could import scipy.stats and use scipy.stats.entropy(doc_freq, summary_freq)
        but this gives equivalent value without the import
        """
        sum_val = 0
        for w in summary_freq:
            frequency = doc_freq.get(w)
            if frequency:  # missing or zero = no frequency
                sum_val += frequency * math.log(frequency / summary_freq[w])

        return sum_val

    @staticmethod
    def _find_index_of_best_sentence(kls):
        """
        the best sentence is the one with the smallest kl_divergence
        """
        return kls.index(min(kls))

    def _compute_ratings(self, sentences):
        word_freq = self.compute_tf(sentences)
        ratings = {}
        summary = []
        sentences_copy = sentences.copy()

        # get all content words once for efficiency
        sentences_as_words = [self._get_content_words_in_sentence(s) for s in sentences]

        # Removes one sentence per iteration by adding to summary
        while len(sentences_copy) > 0:
            # will store all the kls values for this pass
            kls = []

            # converts summary to word list
            summary_as_word_list = self._get_all_words_in_doc(summary)

            for s in sentences_as_words:
                # calculates the joint frequency through combining the word lists
                joint_freq = self._joint_freq(s, summary_as_word_list)

                # adds the calculated kl divergence to the list in index = sentence used
                kls.append(self._kl_divergence(joint_freq, word_freq))

            # to consider and then add it into the summary
            index_to_remove = self._find_index_of_best_sentence(kls)
            best_sentence = sentences_copy.pop(index_to_remove)
            del sentences_as_words[index_to_remove]
            summary.append(best_sentence)

            # value is the iteration in which it was removed multiplied by -1 so that
            # the first sentences removed (the most important) have highest values
            ratings[best_sentence] = -1 * len(ratings)

        return ratings

    def _get_best_sentences(self, ratings, sentences):
        k = round(self.summary_part * len(sentences))
        sorted_sentences = list(ratings.keys())[:k]

        return " ".join(sorted_sentences)

In [16]:
KLD = KLSummarizer(rus_stopwords)

def predict_kld(text, summary):
    return KLD(text)

In [ ]:
calc_method_score(gazeta_v1_dataset["test"], predict_kld)

  0%|          | 0/5770 [00:00<?, ?it/s]

0it [00:00, ?it/s]

-------------METRICS-------------
Count:	 5770
Ref:	 россия пригласила китай принять участие в тендере на строительство нового класса атомного ледокола , что обязательно потребует разработки реакторов — и пекин воспользуется этим для создания собственного атомного авианосца . по данным аналитиков , китайцы хотят спустить на воду авианосец , сопоставимый по размерам и характеристикам с американскими кораблями типа gerald r . ford .
Hyp:	 эксперт считает , что у россии есть технологии , но нет денег , а у китая — обратная ситуация . корабль , созданный по примеру своего предшественника liaoning , является вторым авианосцем в китае . по мнению китайских экспертов , чтобы быть конкурентоспособным , пекину необходим более мощный атомный авианосец .
BLEU:     	3.5
chrF:     	28.7
ROUGE-1-F:	19.0
ROUGE-2-F:	4.7
ROUGE-L-F:	15.9
METEOR:   	14.0
Dup 1-grams:	26.6
Dup 2-grams:	3.7
Dup 3-grams:	1.0
Avg length:	394.8


In [ ]:
calc_method_score(gazeta_v2_dataset["test"], predict_kld)

  0%|          | 0/6793 [00:00<?, ?it/s]

0it [00:00, ?it/s]

-------------METRICS-------------
Count:	 6793
Ref:	 токио пока не дал гарантий москве , что не станет размещать на своей территории американские ракеты , заявил владимир путин . это , по его словам , препятствует заключению мирного соглашения с японией . его отсутствие президент назвал нонсенсом .
Hyp:	 путин подчеркнул , что и россия , и япония заинтересованы в « абсолютной » нормализации двусторонних отношений , поскольку у стран есть интерес для дальнейшего развития стратегического сотрудничества . тогда , по словам премьера , они были нацелены на снижение налоговой нагрузки , а новая зона должна привлечь инвесторов на курилы . трутнев заявлял , что офшор позволит « поставить точку » в вопросах принадлежности курильских островов .
BLEU:     	3.3
chrF:     	28.5
ROUGE-1-F:	18.5
ROUGE-2-F:	4.4
ROUGE-L-F:	15.4
METEOR:   	13.5
Dup 1-grams:	26.8
Dup 2-grams:	3.9
Dup 3-grams:	1.1
Avg length:	398.3


## Модели машинного и глубокого обучения

### Extractive summarization with word2vec

про word2vec - https://github.com/Urezzzer/deep_learning_school_mipt_fall_2021/blob/main/second_semester/word2vec.ipynb

Читай про LexRank, тоже самое, но используются другие векторные представления предложений
https://cyberleninka.ru/article/n/obzor-zadachi-avtomaticheskoy-summarizatsii-teksta/viewer

https://iq.opengenus.org/lexrank-text-summarization/

https://habr.com/ru/post/514540/ - Экстрактивная суммаризация на основе обученных векторных представлений

#### Code

In [ ]:
import collections
from navec import Navec
from sklearn.metrics.pairwise import cosine_similarity

path = '/content/navec_hudlit_v1_12B_500K_300d_100q.tar'
navec = Navec.load(path)

class LexRankSummarizer_Word2Vec():

    def __init__(self, stopwords, vectors=navec, summary_part=0.1, pad='<pad>'):
        self._stop_words = frozenset(stopwords)
        self.epsilon = 1e-4
        self.threshold = 0.1
        self.summary_part = summary_part
        self.vectors = vectors
        self.pad = pad

    def stop_words(self):
        return self._stop_words

    def __call__(self, document):

        sentences_words = [[morph.parse(token.text)[0].normal_form for token in razdel.tokenize(sentence.text) if token.text.lower() not in self._stop_words] for sentence in razdel.sentenize(document)]
        sentences = [sentence.text for sentence in razdel.sentenize(document)]

        embed_sentences = self._sent2vec(sentences_words)
        matrix = self._create_matrix(sentences_words, self.threshold, embed_sentences)
        ranks = self.power_method(matrix, self.epsilon)
        
        return self._get_best_sentences({i: rank for i, rank in enumerate(ranks)}, sentences)

    def _sent2vec(self, sentences):
        embed_sentences = {}
        for i, sentence in enumerate(sentences):
            emb_sentence = np.zeros(300)
            for word in sentence:
                try:
                    token = self.vectors[word]
                except:
                    token = self.vectors[self.pad]
                emb_sentence += token
            emb_sentence /= len(sentence)
            embed_sentences[i] = emb_sentence
        return embed_sentences

    def _create_matrix(self, sentences, threshold, embed_sentences):
        """
        Creates matrix of shape |sentences|×|sentences|.
        """
        # create matrix |sentences|×|sentences| filled with zeroes
        sentences_count = len(sentences)
        matrix = np.zeros((sentences_count, sentences_count))
        degrees = np.zeros((sentences_count, ))

        for row, sentence1 in enumerate(sentences):
            for col, sentence1 in enumerate(sentences):
                matrix[row, col] = cosine_similarity(embed_sentences[row].reshape(1, -1), embed_sentences[col].reshape(1, -1))
                if matrix[row, col] > threshold:
                    matrix[row, col] = 1.0
                    degrees[row] += 1
                else:
                    matrix[row, col] = 0

        for row in range(sentences_count):
            for col in range(sentences_count):
                if degrees[row] == 0:
                    degrees[row] = 1

                matrix[row][col] = matrix[row][col] / degrees[row]

        return matrix

    @staticmethod
    def power_method(matrix, epsilon):
        transposed_matrix = matrix.T
        sentences_count = len(matrix)
        p_vector = np.array([1.0 / sentences_count] * sentences_count)
        lambda_val = 1.0

        while lambda_val > epsilon:
            next_p = np.dot(transposed_matrix, p_vector)
            lambda_val = np.linalg.norm(np.subtract(next_p, p_vector))
            p_vector = next_p

        return p_vector
    
    def _get_best_sentences(self, ratings, sentences):
        k = round(self.summary_part * len(sentences))
        sorted_ind = sorted([i for i, _ in sorted(ratings.items(), key=lambda item: item[1], reverse=True)][:k])
        return " ".join([sentences[i] for i in sorted_ind])

#### Implementation

In [ ]:
LexRank_W2V = LexRankSummarizer_Word2Vec(rus_stopwords)

def predict_lex_rank_word2vec(text, summary):
    return LexRank_W2V(text)

In [ ]:
calc_method_score(gazeta_v1_dataset["test"], predict_lex_rank_word2vec)

  0%|          | 0/5770 [00:00<?, ?it/s]

0it [00:00, ?it/s]

-------------METRICS-------------
Count:	 5770
Ref:	 россия пригласила китай принять участие в тендере на строительство нового класса атомного ледокола , что обязательно потребует разработки реакторов — и пекин воспользуется этим для создания собственного атомного авианосца . по данным аналитиков , китайцы хотят спустить на воду авианосец , сопоставимый по размерам и характеристикам с американскими кораблями типа gerald r . ford .
Hyp:	 конечно , ничего нельзя сказать определенно до тех пор , пока первый ядерный носитель китая фактически не вступит в строй — где-то в 2030 году » , — указывает аналитик . строительство первого китайского авианосца собственного производства началось еще в ноябре 2013 года . судно , названное , предположительно , в честь рака-богомола pipixia , должно заступить на боевое дежурство не ранее 2020 года .
BLEU:     	6.1
chrF:     	32.8
ROUGE-1-F:	24.8
ROUGE-2-F:	8.8
ROUGE-L-F:	20.2
METEOR:   	20.6
Dup 1-grams:	22.3
Dup 2-grams:	2.3
Dup 3-grams:	0.6
Avg length:

In [ ]:
calc_method_score(gazeta_v2_dataset["test"], predict_lex_rank_word2vec)

  0%|          | 0/6793 [00:00<?, ?it/s]

0it [00:00, ?it/s]

-------------METRICS-------------
Count:	 6793
Ref:	 токио пока не дал гарантий москве , что не станет размещать на своей территории американские ракеты , заявил владимир путин . это , по его словам , препятствует заключению мирного соглашения с японией . его отсутствие президент назвал нонсенсом .
Hyp:	 москва никогда не отказывалась обсуждать с токио мирный договор , который так и не был заключен между двумя странами по итогам второй мировой войны . об этом заявил президент владимир путин на пленарном заседании восточного экономического форума ( вэф ) . тогда , по словам премьера , они были нацелены на снижение налоговой нагрузки , а новая зона должна привлечь инвесторов на курилы .
BLEU:     	5.9
chrF:     	33.4
ROUGE-1-F:	24.7
ROUGE-2-F:	8.3
ROUGE-L-F:	20.2
METEOR:   	20.0
Dup 1-grams:	21.3
Dup 2-grams:	2.1
Dup 3-grams:	0.5
Avg length:	511.3
